In [3]:
#Import libraries
import os
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import json
import csv
import earthaccess


In [4]:
#Login
earthaccess.login(persist=True)

In [5]:
#Set desired time and location

field = gp.read_file('./Isla_Vista_Kelp.geojson')
bbox = tuple(list(field.total_bounds))
bbox #Display coordinate bounds
with open('./Isla_Vista_Kelp.geojson', 'r') as f:
    data = json.load(f)
# Extract the name
location = (data['name']).replace('.kmz', '').replace(' ', '_')
temporal = ("2017-06-01T00:00:00", "2018-01-01T00:00:00") #
location

'Isla_Vista_Kelp'

In [10]:
#Search for satellite data from  Landsat 30m and Sentinel 30m
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bbox,
    temporal=temporal,
    cloud_cover=.05,
    count=-1
)

dem_results = earthaccess.search_data(
    short_name="ASTGTM",
    bounding_box=bbox)

print(results[0])
print(dem_results[0])

Granules found: 84
Granules found: 1
Collection: {'EntryTitle': 'HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -120.25585253, 'Latitude': 34.20967567}, {'Longitude': -119.06537353, 'Latitude': 34.23552162}, {'Longitude': -119.09014717, 'Latitude': 35.2250277}, {'Longitude': -120.29518354, 'Latitude': 35.19820772}, {'Longitude': -120.25585253, 'Latitude': 34.20967567}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2017-06-06T18:52:54.820Z', 'EndingDateTime': '2017-06-06T18:52:54.820Z'}}
Size(MB): 218.70975589752197
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2017157T183921.v2.0/HLS.S30.T11SKU.2017157T183921.v2.0.Fmask.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2017157T183921.v2.0/HLS.S30.T11SKU.2017157T183921.v2.0.B

In [14]:
folder_path = os.path.join(os.getcwd(),(f'imagery/{location}'))
if not os.path.isfile(os.path.join(folder_path, dem_name)):
        dem_path = earthaccess.download(dem_results[0], local_path=folder_path)
        os.rename(dem_path[0], os.path.join(folder_path,'dem.tif'))
        os.rename(dem_path[1], os.path.join(folder_path, 'num.tif'))
if not os.path.isdir(folder_path):
    os.mkdir(folder_path)
for i, granule in enumerate(results):
    if i > 2:
        break
    json_str = json.dumps(granule.__dict__)
    metadata = json.loads(json_str) 
    meta = metadata['render_dict']['meta']
    name = meta['native-id']

    #For some reason, attributes are parsed into a list in the HLS metadata. This reformats it into a dictionary.
    attributes_list = metadata['render_dict']['umm']['AdditionalAttributes']

    attributes = {attr['Name']: attr['Values'][0] for attr in attributes_list}
    if(int(attributes['CLOUD_COVERAGE']) < 10): #Reject granules with large cloud cover, for now
        continue
    time = attributes['SENSING_TIME']

    folder_name = (f'{name}')
    file_path = os.path.join(folder_path,folder_name)
    if not os.path.isdir(file_path):
        os.mkdir(file_path) #Make folder for granule 

    downloadPath = earthaccess.download(granule, local_path=file_path)
    csv_file = os.path.join(file_path, (f'{folder_name}_metadata.csv'))
    metadata_full_dict = {**attributes, **meta}
    metadata_full_dict['data_vis_url'] = granule.dataviz_links()

    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(metadata_full_dict.keys())
        writer.writerow(metadata_full_dict.values())




 Getting 1 granules, approx download size: 0.02 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

 Getting 1 granules, approx download size: 0.21 GB


...

 Getting 1 granules, approx download size: 0.17 GB


QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

File HLS.L30.T10SGD.2017167T183413.v2.0.B06.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B09.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B05.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B01.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.VAA.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B04.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B11.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.Fmask.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B02.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.VZA.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.SZA.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.SAA.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B07.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B03.tif already downloaded
File HLS.L30.T10SGD.2017167T183413.v2.0.B10.tif already down

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]